# Importing the libraries

In [ ]:
!pip install --force-reinstall numpy==1.26.4
!pip install --force-reinstall seaborn==0.13.2
!pip install shap lime xgboost category_encoders joblib


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 22.1 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opencv-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
pytensor 2.35.1 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
opencv-python-headless 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
jaxlib 0.7.2 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
jax 0.7.2 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
shap 0.50.0 

  Using cached numpy-2.3.5-cp312-cp312-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (62 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.3/112.3 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 34.6 MB/s eta 0:00:00
Using cached numpy-2.3.5-cp312-cp312-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl (16.6 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 38.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.6/362.6 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 29.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 41.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 45.6 MB/s eta 0:00:00
   ━━━━━━━━━━━

# Install the required packages

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

from sklearn.metrics import (
    roc_auc_score, f1_score, precision_score, recall_score,
    confusion_matrix, classification_report
)

import xgboost as xgb
import shap
from lime.lime_tabular import LimeTabularExplainer
import joblib
import os


# Load the dataset

In [ ]:
from google.colab import drive

drive.flush_and_unmount()

drive.mount('/content/drive')

Mounted at /content/drive


In [16]:
# ---------------------------------------
# SAVE ALL OUTPUTS TO GOOGLE DRIVE
# ---------------------------------------
output_dir = "/content/drive/MyDrive/churn_outputs"
os.makedirs(output_dir, exist_ok=True)

# Save model
joblib.dump(pipeline, f"{output_dir}/model.joblib")

# Save SHAP Summary Plot
plt.savefig(f"{output_dir}/shap_summary.png", bbox_inches="tight")
plt.close()

# Save LIME explanation
exp.save_to_file(f"{output_dir}/lime_example.html")



In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Telco_customer_churn.csv")
df.head()


,CustomerID,Count,Country,State,City,Zip Code,Lat Long,Latitude,Longitude,Gender,...,Contract,Paperless Billing,Payment Method,Monthly Charges,Total Charges,Churn Label,Churn Value,Churn Score,CLTV,Churn Reason
0,3668-QPYBK,1,United States,California,Los Angeles,90003,"33.964131, -118.272783",33.964131,-118.272783,Male,...,Month-to-month,Yes,Mailed check,53.85,108.15,Yes,1,86,3239,Competitor made better offer
1,9237-HQITU,1,United States,California,Los Angeles,90005,"34.059281, -118.30742",34.059281,-118.307420,Female,...,Month-to-month,Yes,Electronic check,70.70,151.65,Yes,1,67,2701,Moved
2,9305-CDSKC,1,United States,California,Los Angeles,90006,"34.048013, -118.293953",34.048013,-118.293953,Female,...,Month-to-month,Yes,Electronic check,99.65,820.5,Yes,1,86,5372,Moved
3,7892-POOKP,1,United States,California,Los Angeles,90010,"34.062125, -118.315709",34.062125,-118.315709,Female,...,Month-to-month,Yes,Electronic check,104.80,3046.05,Yes,1,84,5003,Moved
4,0280-XJGEX,1,United States,California,Los Angeles,90015,"34.039224, -118.266293",34.039224,-118.266293,Male,...,Month-to-month,Yes,Bank transfer (automatic),103.70,5036.3,Yes,1,89,5340,Competitor had better devices


In [ ]:
# -----------------------------
# 1. Clean column names
# -----------------------------
df.columns = (
    df.columns
    .str.replace(" ", "_")
    .str.replace("-", "_")
)

# Now names look like: Total_Charges, Monthly_Charges, Churn_Value

# -----------------------------
# 2. Standardize important column names
# -----------------------------
rename_map = {
    "Total_Charges": "TotalCharges",
    "Monthly_Charges": "MonthlyCharges",
    "Churn_Value": "Churn",
    "Churn_Label": "ChurnLabel",
    "Payment_Method": "PaymentMethod",
    "Paperless_Billing": "PaperlessBilling"
}

df.rename(columns=rename_map, inplace=True)

# -----------------------------
# 3. Convert numbers safely
# -----------------------------
if "TotalCharges" in df.columns:
    df["TotalCharges"] = pd.to_numeric(df["TotalCharges"], errors="coerce")

# -----------------------------
# 4. Drop ID columns
# -----------------------------
drop_cols = ["CustomerID", "Lat_Long", "ChurnLabel", "Churn_Reason"]
df = df.drop([c for c in drop_cols if c in df.columns], axis=1)

# -----------------------------
# 5. Define target variable
# -----------------------------
if "Churn" not in df.columns:
    raise ValueError("Target column 'Churn Value' (renamed to 'Churn') not found!")

y = df["Churn"]
X = df.drop("Churn", axis=1)

df.head()


,Count,Country,State,City,Zip_Code,Latitude,Longitude,Gender,Senior_Citizen,Partner,...,Streaming_TV,Streaming_Movies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn,Churn_Score,CLTV
0,1,United States,California,Los Angeles,90003,33.964131,-118.272783,Male,No,No,...,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,1,86,3239
1,1,United States,California,Los Angeles,90005,34.059281,-118.307420,Female,No,No,...,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,1,67,2701
2,1,United States,California,Los Angeles,90006,34.048013,-118.293953,Female,No,No,...,Yes,Yes,Month-to-month,Yes,Electronic check,99.65,820.50,1,86,5372
3,1,United States,California,Los Angeles,90010,34.062125,-118.315709,Female,No,Yes,...,Yes,Yes,Month-to-month,Yes,Electronic check,104.80,3046.05,1,84,5003
4,1,United States,California,Los Angeles,90015,34.039224,-118.266293,Male,No,No,...,Yes,Yes,Month-to-month,Yes,Bank transfer (automatic),103.70,5036.30,1,89,5340


# Train/Test Split

In [ ]:
num_features = X.select_dtypes(include=["int64", "float64"]).columns
cat_features = X.select_dtypes(include=["object"]).columns

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)


# Preprocessing Pipeline

In [ ]:
numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, num_features),
        ("cat", categorical_transformer, cat_features)
    ]
)


# XGBoost + Randomized Search

In [ ]:
model = xgb.XGBClassifier(
    objective="binary:logistic",
    eval_metric="logloss",
)

param_grid = {
    "n_estimators": [100, 200, 300],
    "max_depth": [3, 4, 5, 6],
    "learning_rate": [0.01, 0.05, 0.1],
    "subsample": [0.7, 0.8, 1.0],
    "colsample_bytree": [0.7, 0.8, 1.0]
}

clf = RandomizedSearchCV(
    estimator=model,
    param_distributions=param_grid,
    scoring="roc_auc",
    cv=3,
    n_iter=10,
    verbose=1,
    n_jobs=-1
)


# Train Pipeline

In [ ]:
pipeline = Pipeline(steps=[
    ("preprocess", preprocessor),
    ("model", clf)
])

pipeline.fit(X_train, y_train)


Fitting 3 folds for each of 10 candidates, totalling 30 fits


Pipeline(steps=[('preprocess',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  Index(['Count', 'Zip_Code', 'Latitude', 'Longitude', 'Tenure_Months',
       'MonthlyCharges', 'TotalCharges', 'Churn_Score', 'CLTV'],
      dtype='object')),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strate...
                                                            max_leaves=None,
                                                            min_child_weight=None,
                                                            missing=nan,
                                                            monotone_constraints=None,
                                                            multi_strategy=None,
                                                            n_estimators=None,
                                                            n_jobs=None,
                                                            num_parallel_tree=None, ...),
                                    n_jobs=-1,
                                    param_distributions={'colsample_bytree': [0.7,
                                                                              0.8,
                                                                              1.0],
                                                         'learning_rate': [0.01,
                                                                           0.05,
                                                                           0.1],
                                                         'max_depth': [3, 4, 5,
                                                                       6],
                                                         'n_estimators': [100,
                                                                          200,
                                                                          300],
                                                         'subsample': [0.7, 0.8,
                                                                       1.0]},
                                    scoring='roc_auc', verbose=1))])

# Evaluate

In [ ]:
y_pred = pipeline.predict(X_test)
y_proba = pipeline.predict_proba(X_test)[:, 1]

print("ROC-AUC:", roc_auc_score(y_test, y_proba))
print("F1:", f1_score(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred))
print("Recall:", recall_score(y_test, y_pred))

print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))


ROC-AUC: 0.9832132062311092
F1: 0.8688741721854305
Precision: 0.8608923884514436
Recall: 0.8770053475935828

Confusion Matrix:
 [[982  53]
 [ 46 328]]

Classification Report:
               precision    recall  f1-score   support

           0       0.96      0.95      0.95      1035
           1       0.86      0.88      0.87       374

    accuracy                           0.93      1409
   macro avg       0.91      0.91      0.91      1409
weighted avg       0.93      0.93      0.93      1409



# Save Outputs

In [ ]:
os.makedirs("/content/churn_outputs", exist_ok=True)
joblib.dump(pipeline, "/content/churn_outputs/model.joblib")


['/content/My Drive/churn_outputs/model.joblib']

# SHAP (New API)

In [ ]:
best_model = pipeline.named_steps["model"].best_estimator_

X_train_processed = pipeline.named_steps["preprocess"].transform(X_train)

# Convert sparse matrix to dense array if it's sparse
if hasattr(X_train_processed, 'toarray'):
    X_train_processed = X_train_processed.toarray()

# Get feature names after preprocessing
num_feature_names = num_features.tolist()
cat_feature_names = pipeline.named_steps["preprocess"].named_transformers_['cat'].named_steps['onehot'].get_feature_names_out(cat_features)
all_feature_names = num_feature_names + list(cat_feature_names)

explainer = shap.Explainer(best_model, X_train_processed, feature_names=all_feature_names)
shap_values = explainer(X_train_processed)

shap.summary_plot(shap_values, X_train_processed, feature_names=all_feature_names, show=False)
plt.savefig("/content/churn_outputs/shap_summary.png", bbox_inches="tight")
plt.close()


100%|===================| 5611/5634 [00:31<00:00]       

In [ ]:
# Re-using X_train_processed from the previous cell, which is already a dense array.
# X_train_processed was defined and converted to a dense numpy array in the SHAP cell (LnjYo-5iNxlP).
# So we can directly use it here.
X_train_arr = X_train_processed

# Get all feature names after preprocessing
all_feature_names = pipeline.named_steps["preprocess"].get_feature_names_out()

lime_explainer = LimeTabularExplainer(
    training_data=X_train_arr,
    feature_names=all_feature_names,
    class_names=["No Churn", "Churn"],
    mode="classification"
)

instance = X_test.iloc[0:1]
instance_enc = pipeline.named_steps["preprocess"].transform(instance)

# Convert instance_enc (which is a sparse matrix of shape (1, features)) to a dense 1D array
# LIME expects a 1D array for a single instance
if hasattr(instance_enc, 'toarray'):
    instance_to_explain = instance_enc.toarray()[0] # Convert to dense array, then take the first (and only) row
else:
    instance_to_explain = instance_enc[0] # If it's already a dense numpy array, just take the first row

# Define a custom predict_proba function for LIME that takes preprocessed data
# and passes it directly to the best_model.
def predict_proba_func(data):
    return best_model.predict_proba(data)

exp = lime_explainer.explain_instance(
    instance_to_explain,
    predict_proba_func, # Use the custom predict function that bypasses pipeline's preprocessor
    num_features=10
)

exp.save_to_file("/content/churn_outputs/lime_example.html")